In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-222036
Azure region: southcentralus
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-222036


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "udacityprojectos"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
# cluster_basic = AmlCompute(
#     name="basic-example",
#     type="amlcompute",
#     size="Standard_D2_V2",
#     location="westus",
#     min_instances=0,
#     max_instances=4,
#     idle_time_before_scale_down=120,
# )
compute_target.get_status()


In [6]:

compute_target.get_status().serialize()

{'currentNodeCount': 0,
 'targetNodeCount': 0,
 'nodeStateCounts': {'preparingNodeCount': 0,
  'runningNodeCount': 0,
  'idleNodeCount': 0,
  'unusableNodeCount': 0,
  'leavingNodeCount': 0,
  'preemptedNodeCount': 0},
 'allocationState': 'Steady',
 'allocationStateTransitionTime': '2023-01-09T09:49:37.557000+00:00',
 'errors': None,
 'creationTime': '2023-01-09T09:49:32.868964+00:00',
 'modifiedTime': '2023-01-09T09:49:39.693922+00:00',
 'provisioningState': 'Succeeded',
 'provisioningStateTransitionTime': None,
 'scaleSettings': {'minNodeCount': 0,
  'maxNodeCount': 4,
  'nodeIdleTimeBeforeScaleDown': 'PT1800S'},
 'vmPriority': 'Dedicated',
 'vmSize': 'STANDARD_D2_V2'}

In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling( #random
    {
        '--C': choice(0.01, 0.1, 1, 10, 100, 1000),
        '--max_iter': choice(5, 50, 100)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=compute_target,
    environment=sklearn_env
)### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4
) ### YOUR CODE HERE ###

In [10]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(
    config=hyperdrive_config,
    show_output=True
)
RunDetails(hyperdrive_run).show()
print('finish')
hyperdrive_run.wait_for_completion(show_output=True)



_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

finish
RunId: HD_d723ee76-9a6e-4ff3-81f0-17c5972a475c
Web View: https://ml.azure.com/runs/HD_d723ee76-9a6e-4ff3-81f0-17c5972a475c?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-222036/workspaces/quick-starts-ws-222036&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-01-09T10:46:40.053031][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-01-09T10:46:40.8058063Z][SCHEDULER][INFO]Scheduling job, id='HD_d723ee76-9a6e-4ff3-81f0-17c5972a475c_0' 
[2023-01-09T10:46:40.9071653Z][SCHEDULER][INFO]Scheduling job, id='HD_d723ee76-9a6e-4ff3-81f0-17c5972a475c_1' 
[2023-01-09T10:46:41.0296312Z][SCHEDULER][INFO]Scheduling job, id='HD_d723ee76-9a6e-4ff3-81f0-17c5972a475c_2' 
[2023-01-09T10:46:41.0855391Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_d723ee76-9a6e-4ff3-81f0-17c5972a475c_0' 
[2023-01-09T10:46:41.1798958Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_d723ee76-9a6e-4ff

{'runId': 'HD_d723ee76-9a6e-4ff3-81f0-17c5972a475c',
 'target': 'udacityprojectos',
 'status': 'Completed',
 'startTimeUtc': '2023-01-09T10:46:39.38307Z',
 'endTimeUtc': '2023-01-09T10:52:15.027249Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'd9217dda-f1d0-43cd-abde-371ffd4057f1',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1022-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': '18',
  'score': '0.9116240573845871',
  'best_child_run_id': 'HD_d723ee76-9a6e-4ff3-81f0-17c5972a475c_1',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_d723ee76-9a6e-4ff3-81f0-17c5972a475c_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlCli

KeyError: 'log_files'

In [15]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_hd_run=hyperdrive_run.get_best_run_by_primary_metric()
best_hd_run_metrics = best_hd_run.get_metrics()
hd_parameter_values = best_hd_run.get_details()['runDefinition']['arguments']

print('best_hd_run_metrics: ', best_hd_run_metrics)
print( '\nhd_parameter_values: ', hd_parameter_values, '\n\n')
print(best_hd_run.get_file_names())
#joblib.dump(best_hd_run, 'bestrun.joblib')

best_hd_run_metrics:  {'Regularization Strength:': 1.0, 'Max iterations:': 50, 'Accuracy': 0.9116240573845871}

hd_parameter_values:  ['--C', '1', '--max_iter', '50'] 


['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2023-01-09-10', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [16]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds=TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

train_data, test_data = ds.random_split(percentage=0.77)
print(train_data)

TabularDataset
{
  "source": [
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes",
    "TakeSample"
  ]
}


In [18]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(train_data)
### YOUR DATA OBJECT HERE ###)

In [21]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
train_data_clean = x.join(y)

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data_clean,
    label_column_name='y',
    n_cross_validations=3)

In [22]:
# Submit your automl run

### YOUR CODE HERE ###
automl_exp = Experiment(ws, 'automl_experiment')
automl_run = automl_exp.submit(
    config=automl_config,
    show_output=True
)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
automl_experiment,AutoML_031411bf-a29c-482f-a831-c2d353e93605,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2023/01/09 11:27:51 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023-01-09:11:51:12,636 INFO     [explanation_client.py:334] Using default datastore for uploads


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl_experiment,AutoML_031411bf-a29c-482f-a831-c2d353e93605,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2836                          |1                               |25338                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_031411bf-a29c-482f-a831-c2d353e93605',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2023-01-09T11:16:46.40594Z',
 'endTimeUtc': '2023-01-09T11:49:24.728547Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_experiment","subscription_id":"81cefad3-d2c9-4f77-a466-99a7f541c7bb","resource_group":"aml-quickstarts-222036","workspace_name":"quick-starts-ws-222036","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1

In [25]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_aml_run, best_aml_model = automl_run.get_output()

print(best_aml_run, best_aml_model)

#joblib.dump(best_aml_run, 'best_aml_run.joblib')

joblib.dump(best_aml_model, 'best_aml_model.joblib')

Run(Experiment: automl_experiment,
Id: AutoML_031411bf-a29c-482f-a831-c2d353e93605_30,
Type: None,
Status: Completed) Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('24', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), ran

['best_aml_model.joblib']